# Лабораторная работа 5 (Gradient Boosting)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from utils import regression_cross_validate, display_metrics_table, classification_cross_validate, display_metrics_classification_table
import warnings
warnings.filterwarnings("ignore")

### Regression

#### 1. Обработка данных

In [2]:
df = pd.read_csv('data/laptop_prices.csv')
df.head()

,Company,Product,TypeName,Inches,Ram,OS,Weight,Price_euros,Screen,ScreenW,...,RetinaDisplay,CPU_company,CPU_freq,CPU_model,PrimaryStorage,SecondaryStorage,PrimaryStorageType,SecondaryStorageType,GPU_company,GPU_model
0,Apple,MacBook Pro,Ultrabook,13.3,8,macOS,1.37,1339.69,Standard,2560,...,Yes,Intel,2.3,Core i5,128,0,SSD,No,Intel,Iris Plus Graphics 640
1,Apple,Macbook Air,Ultrabook,13.3,8,macOS,1.34,898.94,Standard,1440,...,No,Intel,1.8,Core i5,128,0,Flash Storage,No,Intel,HD Graphics 6000
2,HP,250 G6,Notebook,15.6,8,No OS,1.86,575.00,Full HD,1920,...,No,Intel,2.5,Core i5 7200U,256,0,SSD,No,Intel,HD Graphics 620
3,Apple,MacBook Pro,Ultrabook,15.4,16,macOS,1.83,2537.45,Standard,2880,...,Yes,Intel,2.7,Core i7,512,0,SSD,No,AMD,Radeon Pro 455
4,Apple,MacBook Pro,Ultrabook,13.3,8,macOS,1.37,1803.60,Standard,2560,...,Yes,Intel,3.1,Core i5,256,0,SSD,No,Intel,Iris Plus Graphics 650


In [3]:
features, target = df.drop(columns=['Price_euros']), df['Price_euros']

numerical_features = features.select_dtypes(include=[np.number]).columns  # Численные признаки
categorical_features = features.select_dtypes(exclude=[np.number]).columns  # Категориальные признаки

train_features, test_features, train_target, test_target = train_test_split(features, target, random_state=42)

le = OrdinalEncoder(handle_unknown='use_encoded_value',
                    unknown_value=-1)

train_features[categorical_features] = le.fit_transform(train_features[categorical_features])
test_features[categorical_features] = le.transform(test_features[categorical_features])

imputer = SimpleImputer(strategy='most_frequent') 
train_features = pd.DataFrame(imputer.fit_transform(train_features), columns=train_features.columns)
test_features = pd.DataFrame(imputer.transform(test_features), columns=test_features.columns)

#### 2. Построение бейзлайна

In [4]:
metrics = regression_cross_validate(GradientBoostingRegressor, train_features.to_numpy(), train_target.to_numpy(), n_folds=5, random_state=42, n_estimators=50, max_depth=10)
display_metrics_table(*metrics)

gb = GradientBoostingRegressor(n_estimators=50, max_depth=10, random_state=42)
gb.fit(train_features, train_target)

predicted_target = gb.predict(test_features)

# Метрики
mse = mean_squared_error(test_target, predicted_target)
mae = mean_absolute_error(test_target, predicted_target)
r2 = r2_score(test_target, predicted_target)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")


| Metric   |        Mean |       Std Dev |
|:---------|------------:|--------------:|
| MAE      |   186.816   |    12.5959    |
| MSE      | 94982.7     | 23122.7       |
| R2       |     0.80653 |     0.0433616 |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 126493.13
Средняя абсолютная ошибка (MAE): 219.29
Коэффициент детерминации (R^2): 0.74


Как можем увидеть значение метрики $R^2$ около 0.74, что означает что около 74% дисперсии данных объясняется моделью.

#### 3. Формулировка гипотез

Сформулируем несколько гипотез, которые могут помочь улучшить качество модели

1) Поменять Encoder категориальных признаков с `LabelEncoder` на `OneHotEncoder`
2) Отмасштабировать численные признаки
3) Уменьшить глубину и  увеличить число деревьев


In [5]:
onehot = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

encoded_train_data = onehot.fit_transform(train_features[categorical_features])
encoded_test_data = onehot.transform(test_features[categorical_features])

encoded_df = pd.DataFrame(encoded_train_data, columns=onehot.get_feature_names_out(categorical_features))
updated_train_features = train_features.drop(columns=categorical_features).reset_index(drop=True)
updated_train_features = pd.concat([updated_train_features, encoded_df], axis=1)

encoded_df = pd.DataFrame(encoded_test_data, columns=onehot.get_feature_names_out(categorical_features))
updated_test_features = test_features.drop(columns=categorical_features).reset_index(drop=True)
updated_test_features = pd.concat([updated_test_features, encoded_df], axis=1)

scaler = StandardScaler()
updated_train_features[numerical_features] = scaler.fit_transform(train_features[numerical_features])
updated_test_features[numerical_features] = scaler.transform(test_features[numerical_features])

In [6]:
metrics = regression_cross_validate(GradientBoostingRegressor, updated_train_features.to_numpy(), train_target.to_numpy(), n_folds=5, random_state=42, n_estimators=100, max_depth=3)
display_metrics_table(*metrics)

gb = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
gb.fit(updated_train_features, train_target)

predicted_target = gb.predict(updated_test_features)

# Метрики
mse = mean_squared_error(test_target, predicted_target)
mae = mean_absolute_error(test_target, predicted_target)
r2 = r2_score(test_target, predicted_target)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |         Mean |       Std Dev |
|:---------|-------------:|--------------:|
| MAE      |   198.144    |    10.3365    |
| MSE      | 92486.7      | 15711.6       |
| R2       |     0.810411 |     0.0362914 |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 63201.39
Средняя абсолютная ошибка (MAE): 193.60
Коэффициент детерминации (R^2): 0.87


Можно заметить, что в среднем показатели метрик значительно улучшились по сравнению с предыдущими результатами. Анализ результатов на тестовой выборке также демонстрирует существенные улучшения, что подтверждает эффективность проведённых модификаций.

#### 4. Реализация своего класса

In [7]:
class MyGradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.05, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []
        self.initial_value = None

    def fit(self, features, target):
        # Initialize model with the mean of target
        self.initial_value = np.mean(target)
        residual = target - self.initial_value

        for _ in range(self.n_estimators):
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(features, residual)
            predictions = tree.predict(features)
            residual -= self.learning_rate * predictions
            self.trees.append(tree)

    def predict(self, features):
        predictions = np.full(features.shape[0], self.initial_value)
        for tree in self.trees:
            predictions += self.learning_rate * tree.predict(features)
        return predictions



In [8]:
metrics = regression_cross_validate(MyGradientBoostingRegressor, train_features.to_numpy(), train_target.to_numpy(), n_folds=5, n_estimators=50, max_depth=10)
display_metrics_table(*metrics)

gb = MyGradientBoostingRegressor(n_estimators=50, max_depth=10)
gb.fit(train_features, train_target)

predicted_target = gb.predict(test_features)

# Метрики
mse = mean_squared_error(test_target, predicted_target)
mae = mean_absolute_error(test_target, predicted_target)
r2 = r2_score(test_target, predicted_target)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")


| Metric   |         Mean |       Std Dev |
|:---------|-------------:|--------------:|
| MAE      |   194.171    |     8.23187   |
| MSE      | 97217.2      | 19480.4       |
| R2       |     0.802034 |     0.0338632 |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 120581.95
Средняя абсолютная ошибка (MAE): 227.92
Коэффициент детерминации (R^2): 0.75


In [9]:
metrics = regression_cross_validate(MyGradientBoostingRegressor, updated_train_features.to_numpy(), train_target.to_numpy(), n_folds=5, n_estimators=300, max_depth=3)
display_metrics_table(*metrics)

gb = GradientBoostingRegressor(n_estimators=100, max_depth=3)
gb.fit(updated_train_features, train_target)

predicted_target = gb.predict(updated_test_features)

# Метрики
mse = mean_squared_error(test_target, predicted_target)
mae = mean_absolute_error(test_target, predicted_target)
r2 = r2_score(test_target, predicted_target)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |         Mean |       Std Dev |
|:---------|-------------:|--------------:|
| MAE      |   193.502    |    10.2628    |
| MSE      | 87898.1      | 13082.6       |
| R2       |     0.819637 |     0.0320388 |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 62954.17
Средняя абсолютная ошибка (MAE): 193.22
Коэффициент детерминации (R^2): 0.87


Результаты демонстрируют, что разработанная собственная реализация модели в целом обеспечивает уровень качества, сопоставимый с результатами, достигаемыми стандартной моделью из библиотеки `sklearn`.

### Classification

 #### 1. Обработка данных

In [10]:
df = pd.read_csv('data/AIDS_Classification.csv')
df.head()

,time,trt,age,wtkg,hemo,homo,drugs,karnof,oprior,z30,...,str2,strat,symptom,treat,offtrt,cd40,cd420,cd80,cd820,infected
0,948,2,48,89.8128,0,0,0,100,0,0,...,0,1,0,1,0,422,477,566,324,0
1,1002,3,61,49.4424,0,0,0,90,0,1,...,1,3,0,1,0,162,218,392,564,1
2,961,3,45,88.4520,0,1,1,90,0,1,...,1,3,0,1,1,326,274,2063,1893,0
3,1166,3,47,85.2768,0,1,0,100,0,1,...,1,3,0,1,0,287,394,1590,966,0
4,1090,0,43,66.6792,0,1,0,100,0,1,...,1,3,0,0,0,504,353,870,782,0


In [11]:
features, target = df.drop(columns=['infected']), df['infected']

train_features, test_features, train_target, test_target = train_test_split(features, target, random_state=42)

num_features = features.select_dtypes(include=[np.number]).columns  # Численные признаки
categorical_features = features.select_dtypes(exclude=[np.number]).columns  # Категориальные признаки

le = OrdinalEncoder(handle_unknown='use_encoded_value',
                    unknown_value=99)

train_features[categorical_features] = le.fit_transform(train_features[categorical_features])
test_features[categorical_features] = le.transform(test_features[categorical_features])

imputer = SimpleImputer(strategy='most_frequent') 
train_features = pd.DataFrame(imputer.fit_transform(train_features), columns=train_features.columns)
test_features = pd.DataFrame(imputer.transform(test_features), columns=test_features.columns)

#### 2. Построение бейзлайна 

In [12]:
metrics = classification_cross_validate(GradientBoostingClassifier, train_features.to_numpy(), train_target.to_numpy(), n_folds=5, n_estimators=50, max_depth=10)
display_metrics_classification_table(*metrics)

gb = GradientBoostingClassifier(random_state=42, n_estimators=50, max_depth=10)
gb.fit(train_features, train_target)

predicted_target = gb.predict(test_features)

accuracy = accuracy_score(test_target, predicted_target)
precision = precision_score(test_target, predicted_target, average='weighted')
recall = recall_score(test_target, predicted_target, average='weighted')
f1 = f1_score(test_target, predicted_target, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |   Std Dev |
|:----------|---------:|----------:|
| Accuracy  | 0.872206 | 0.0129722 |
| Precision | 0.869965 | 0.0132202 |
| Recall    | 0.872206 | 0.0129722 |
| F1-score  | 0.870697 | 0.0131217 |

=== Результаты на Тесте ===
1. Accuracy: 87.66%
2. Precision: 87.44%
3. Recall: 87.66%
4. F1-score: 87.53%


#### 3. Формулировка гипотез

Сформулируем несколько гипотез, которые могут помочь улучшить качество модели

1) Поменять Encoder категориальных признаков с `LabelEncoder` на `OneHotEncoder`
2) Отмасштабировать численные признаки
3) Уменьшить глубину и  увеличить число деревьев

In [13]:
onehot = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

encoded_train_data = onehot.fit_transform(train_features[categorical_features])
encoded_test_data = onehot.transform(test_features[categorical_features])

encoded_df = pd.DataFrame(encoded_train_data, columns=onehot.get_feature_names_out(categorical_features))
updated_train_features = train_features.drop(columns=categorical_features).reset_index(drop=True)
updated_train_features = pd.concat([updated_train_features, encoded_df], axis=1)

encoded_df = pd.DataFrame(encoded_test_data, columns=onehot.get_feature_names_out(categorical_features))
updated_test_features = test_features.drop(columns=categorical_features).reset_index(drop=True)
updated_test_features = pd.concat([updated_test_features, encoded_df], axis=1)


scaler = StandardScaler()
updated_train_features[num_features] = scaler.fit_transform(train_features[num_features])
updated_test_features[num_features] = scaler.transform(test_features[num_features])

In [14]:
metrics = classification_cross_validate(GradientBoostingClassifier, updated_train_features.to_numpy(), train_target.to_numpy(), n_folds=5, random_state=42, n_estimators=100, max_depth=5)
display_metrics_classification_table(*metrics)

gb = GradientBoostingClassifier(n_estimators=100, max_depth=5, random_state=42)
gb.fit(updated_train_features, train_target)

predicted_target = gb.predict(updated_test_features)

accuracy = accuracy_score(test_target, predicted_target)
precision = precision_score(test_target, predicted_target, average='weighted')
recall = recall_score(test_target, predicted_target, average='weighted')
f1 = f1_score(test_target, predicted_target, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |   Std Dev |
|:----------|---------:|----------:|
| Accuracy  | 0.890296 | 0.0252749 |
| Precision | 0.887664 | 0.0270936 |
| Recall    | 0.890296 | 0.0252749 |
| F1-score  | 0.888305 | 0.0264199 |

=== Результаты на Тесте ===
1. Accuracy: 89.53%
2. Precision: 89.32%
3. Recall: 89.53%
4. F1-score: 89.08%


Можно увидеть что в среднем значения метрик улучшились. 
Результаты на тестовой выборке также показывают приросты.

#### 4. Реализация своего класса

In [15]:
class MyGradientBoostingClassifier:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []
        self.initial_value = None

    def _log_odds(self, target):
        p = np.clip(np.mean(target), 1e-15, 1 - 1e-15)
        return np.log(p / (1 - p))

    def fit(self, features, target):
        # Initialize with log-odds
        self.initial_value = self._log_odds(target)
        residual = target - self._sigmoid(self.initial_value)

        for _ in range(self.n_estimators):
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(features, residual)
            predictions = tree.predict(features)
            residual -= self.learning_rate * predictions
            self.trees.append(tree)

    def predict_proba(self, features):
        log_odds = np.full(features.shape[0], self.initial_value)
        for tree in self.trees:
            log_odds += self.learning_rate * tree.predict(features)
        proba = self._sigmoid(log_odds)
        return np.vstack([1 - proba, proba]).T

    def predict(self, features):
        proba = self.predict_proba(features)
        return (proba[:, 1] > 0.3).astype(int)
        # return proba

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

In [16]:
metrics = classification_cross_validate(MyGradientBoostingClassifier, train_features.to_numpy(), train_target.to_numpy(), n_folds=5, n_estimators=5, max_depth=1)
display_metrics_classification_table(*metrics)

gb = MyGradientBoostingClassifier(n_estimators=100, max_depth=10)
gb.fit(train_features, train_target)

predicted_target = gb.predict(test_features)

accuracy = accuracy_score(test_target, predicted_target)
precision = precision_score(test_target, predicted_target, average='weighted')
recall = recall_score(test_target, predicted_target, average='weighted')
f1 = f1_score(test_target, predicted_target, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |   Std Dev |
|:----------|---------:|----------:|
| Accuracy  | 0.756236 | 0.0254717 |
| Precision | 0.572542 | 0.0392092 |
| Recall    | 0.756236 | 0.0254717 |
| F1-score  | 0.651508 | 0.0346697 |

=== Результаты на Тесте ===
1. Accuracy: 85.05%
2. Precision: 84.90%
3. Recall: 85.05%
4. F1-score: 84.97%


In [17]:
metrics = classification_cross_validate(MyGradientBoostingClassifier, updated_train_features.to_numpy(), train_target.to_numpy(), n_folds=5, n_estimators=100, max_depth=5)
display_metrics_classification_table(*metrics)

gb = MyGradientBoostingClassifier(n_estimators=10, max_depth=5)
gb.fit(updated_train_features, train_target)

predicted_target = gb.predict(updated_test_features)

accuracy = accuracy_score(test_target, predicted_target)
precision = precision_score(test_target, predicted_target, average='weighted')
recall = recall_score(test_target, predicted_target, average='weighted')
f1 = f1_score(test_target, predicted_target, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |   Std Dev |
|:----------|---------:|----------:|
| Accuracy  | 0.885929 | 0.0200728 |
| Precision | 0.884384 | 0.020128  |
| Recall    | 0.885929 | 0.0200728 |
| F1-score  | 0.883786 | 0.0201912 |

=== Результаты на Тесте ===
1. Accuracy: 87.48%
2. Precision: 88.18%
3. Recall: 87.48%
4. F1-score: 86.13%


Результаты показывают, что собственная реализация модели в среднем такой же по качеству что и модели из `sklearn`, однако демонстрирует приросты при использовании улучшенного бейзлайна.

### Заключение

Внесённые изменения, включая нормализацию данных и увеличение числа деревьев и уменьшение глубины в алгоритме бустинга, улучшают точность модели. Проведённые эксперименты демонстрируют, что как собственная реализация, так и применение моделей из sklearn, дают схожие результаты.

| Модель                    |      MSE  |        MAE |      $R^2$ |
|:--------------------------|----------:|-----------:|-----------:|
| Sklearn (до улучшения)    | 126493.13 | 219.29     |  0.74      |
| Sklearn (после улучшения) | 63201.39  | 193.60     |  0.87      |
| Собственная имплементация (до улучшения)   | 122053.65   | 228.68    |  0.75      |
| Собственная имплементация (после улучшения)| 62979.43  | 193.22    |  0.87      |

| Модель                    | Accuracy | Precision | Recall | F1-score |
|:--------------------------|----------:|-----------:|--------:|---------:|
| Sklearn (до улучшения)    |  87.66%   |   87.44%   |  87.66% |   87.53% |
| Sklearn (после улучшения) |  89.53%   |   89.32%   |  89.53% |   89.08% |
| Собственная имплементация (до улучшения)   |  85.05%   |   84.83%   |  85.05% |   84.93% |
| Собственная имплементация (после улучшения)|  87.48%   |   88.18%   |  87.48% |   86.13% |